<a href="https://colab.research.google.com/github/nws700f/Python/blob/master/2019_09_19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade -q gspread
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium
!pip install BeautifulSoup
!pip install gspread-dataframe

In [ ]:
import csv
from time import sleep
import datetime
import pandas as pd
import requests
import pathlib
import random
import re
import glob
import openpyxl
import itertools
from time import sleep
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from itertools import zip_longest
from google.colab import auth
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import gspread
from oauth2client.client import GoogleCredentials
import gspread_dataframe as gs_df

# google settings
auth.authenticate_user()
gc = gspread.authorize(GoogleCredentials.get_application_default())
workbook = gc.open_by_url('https://docs.google.com/spreadsheets/d/1efl_wY4PFZOJEGCnu49fuzXZzaF0N8XDXJVZrst_OOU/edit#gid=0')
worksheet_user_list = workbook.worksheet('調査対象ID')
workbook_ochisatsu = gc.open_by_url('https://docs.google.com/spreadsheets/d/1BjGJsAQaOTgKwMtD_VVte_3dLpFinQ5wWx8Z_L-HUvY/edit#gid=0')
worksheet_NG = workbook_ochisatsu.worksheet('NG')

# NG word list
NG_word_list = worksheet_NG.col_values(1)
NG_title_list = worksheet_NG.col_values(2)

# selenium settings
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',options=options)

# パラメータ
border_yahoo = 3000
border_ali = 16666

# スクロール関数(class)
def scroll_class(class_name):
    element = driver.find_element_by_class_name(class_name)
    actions = ActionChains(driver)
    actions.move_to_element(element)
    actions.perform()
    sleep(2)
    #driver.execute_script("window.scrollTo(0, 200);")
# スクロール関数(id)
def scroll_id(id_name):
    element = driver.find_element_by_id(id_name)
    actions = ActionChains(driver)
    actions.move_to_element(element)
    actions.perform()
    sleep(1)
    #driver.execute_script("window.scrollTo(0, 500);")
# スクロール関数(xpath)
def scroll_xpath(xpath):
    element = driver.find_element_by_xpath(xpath)
    actions = ActionChains(driver)
    actions.move_to_element(element)
    actions.perform()
    sleep(1)
    #driver.execute_script("window.scrollTo(0, 500);")

# 数字入力チェック
def input_number():
    try:
        status = int(input())
    except:
        print("Please enter 0 or 1")
        return input_number()
    if status == 0 or status == 1:
        return status
    else:
        print("Please enter 0 or 1")
        return input_number()

# URL 入力チェック
def url_input():
    try:
        url = input()
    except:
        print('Please enter URL')
        return url_input()
    
    if 'http' in url:
        return url
    else:
        print('Please enter URL')
        return url_input()

# 入力
def input_number_multiple():
    try:
        status = int(input())
    except:
        print("Please enter 0 or 1 or 2")
        return input_number()
    if status == 0 or status == 1 or status == 2:
        return status
    else:
        print("Please enter 0 or 1 or 2")
        return input_number()

# オチサツ
def get_info_bid():
    # spread sheet から読みこむ形に変更
    user_list = worksheet_user_list.col_values(1)
    print(user_list)    
    data = []
    #　標準入力 - 年, 月
    print("検索年を入力")
    year = int(input())
    print("検索月を入力")
    month = int(input())
    count = 0
    for user in user_list:
      # if count == 3:
      #   break
      # count += 1
      url = 'http://ochisatsu.com/search/?code=%B4%C1%BB%FA%C8%BD%CA%CC%CD%D1&tl=&id=&u0={}&s=-pr&u1=&d1=&d2=&u2=&p1=&p2=&c=&img=1&{}{}=8%B7%EE'.format(user, year, month);
      driver.get(url)
      driver.implicitly_wait(3)
      # ページ内のデータ取得
      while(True):
          try:
              scroll_xpath('/html/body/center[2]/center/p[1]/a')
              driver.execute_script("window.scrollTo(0, 750);")
              sleep(2)
              element = driver.find_element_by_xpath('/html/body/center[2]/center/p[5]/table')
              t1_list = element.find_elements_by_class_name('t1')
              for t1 in t1_list:
                  link_text = t1.find_element_by_tag_name('a')
                  #print(link_text.text)
                  link = link_text.get_attribute('href')
                  #print(link)
                  link_text = link_text.text
                  # 画像 URL
                  link_image = t1.find_element_by_tag_name('img')
                  image_url = link_image.get_attribute('src')
                  #image_url = image_url.text
                  
                  # データを辞書に登録
                  info_yahoo = {
                      'ベンチマークアカ' : user,
                      'ヤフオクURL' : link,         
                      'ヤフオク商品名' : link_text,
                      'オチサツURL': driver.current_url,
                      'オチサツ画像URL': image_url,
                      'オチサツ画像検索URL': 'https://www.google.co.jp/searchbyimage?image_url=' + image_url + '&encoded_image=&image_content=&as_sitesearch=Aliexpress.com&filename=&hl=ja',
                  }
                  print(info_yahoo)
                  data.append(info_yahoo)
          except:
              link_text = ''
              link = ''
              # データを辞書に登録
              info_yahoo = {
                  'ベンチマークアカ' : user,
                  'ヤフオクURL' : link,         
                  'ヤフオク商品名' : link_text,
                  'オチサツURL': driver.current_url,
                  'オチサツ画像URL': image_url,
                  'オチサツ画像検索URL': '',
              }
              print(info_yahoo)
              data.append(info_yahoo)
          # 次のページをクリック
          try:
              driver.find_element_by_link_text('次のページ').click()
          except:
              break
    
    return year, month, data

# ヤフオク
def get_info_yahoo(data_bid):
    #print(data_bid)
    data_yahoo = []
    for data in data_bid:
        url = data['ヤフオクURL']
        print(url)
        if url != '':
            driver.get(url)
            driver.implicitly_wait(3)
            if driver.find_elements_by_id('modAlertBox'):
                info = {
                    'ヤフオク落札価格' : '',
                    'ヤフオク本品' : '',
                    'ヤフオク送料' : '',
                    'ヤフオク画像検索URL':''
                }
            else:
                # 本体価格
                price = driver.find_element_by_class_name('Price__value')  # 本体価格
                price = price.text                 
                # 税込 or 税別
                price_plane = price.split('税')[0]
                price_plane_revise = int(re.sub('[^0-9]', '', price_plane))
                price_tax = price.split('税')[-1]
                price_tax_include = int(re.sub('[^0-9]', '', price_tax))
                print(price_plane_revise)
                print(price_tax_include)
                if price_tax_include != 0:
                    price = price_tax_include
                else:
                    price = price_plane_revise
                if price < border_yahoo:
                  info = {
                    'ヤフオク落札価格' : '',
                    'ヤフオク本品' : '',
                    'ヤフオク送料' : '',
                    'ヤフオク画像検索URL':''
                  }
                  continue
                # 送料
                scroll_class('Seller__title.sellerInfo')
                sleep(3)
                elem_postage = driver.find_element_by_class_name('Price__postageValue')
                elem_postage = elem_postage.text
                #　画像 URL
                if len(driver.find_elements_by_class_name('ProductImage__thumbnails')) > 0:
                    elem_image = driver.find_element_by_class_name('ProductImage__thumbnails')
                    image_url = elem_image.find_element_by_tag_name('img')
                    image_url = image_url.text
                else:
                    image_url = ''
                # 送料の判定(落札者負担 or 無料 or 全国一律)
                if '無料' in elem_postage:
                    postage = 0
                    info = {
                        'ヤフオク落札価格' : price,
                        'ヤフオク本品' : price,
                        'ヤフオク送料' : postage,
                        'ヤフオク画像検索URL': image_url
                    }
                elif '全国一律' in elem_postage:
                    postage = int(re.sub('[^0-9]','', elem_postage))
                    info = {
                        'ヤフオク落札価格' : price,
                        'ヤフオク本品' : price,
                        'ヤフオク送料' : postage,
                        'ヤフオク画像検索URL': image_url
                    } 
                elif '落札者負担' in elem_postage:
                    try:
                        driver.find_element_by_id('postageDetailBuy').click()
                    except:
                        driver.find_element_by_id('postageDetailCurrent').click()
                    sleep(2)
                    postage = driver.find_element_by_class_name('BidModal__postagePrice')
                    postage = postage.text
                    print('postage:', postage)
                    if '送料未定' in postage:
                        #postage_element = driver.find_element_by_class_name('Select.js-localpost-select')
                        #postage_select_element = Select(postage_element) 
                        postage = 2000
                        info = {
                        'ヤフオク落札価格' : price + postage,
                        'ヤフオク本品' : price,
                        'ヤフオク送料' : postage,
                        'ヤフオク画像検索URL': image_url
                        }
                    elif postage == '':
                        info = {
                            'ヤフオク落札価格' : 'Exception',
                            'ヤフオク本品' : price,
                            'ヤフオク送料' : 'Exception',
                            'ヤフオク画像検索URL': image_url
                            }
                    else:     
                        postage = re.sub('[^0-9]','', postage)
                        postage = int(postage)
                        price_combine = price + postage
                        info = {
                            'ヤフオク落札価格' : price_combine,
                            'ヤフオク本品' : price,
                            'ヤフオク送料' : postage,
                            'ヤフオク画像検索URL': image_url
                        }    
        else:
            info = {
                'ヤフオク落札価格' : '',
                'ヤフオク本品' : '',
                'ヤフオク送料' : '',
                'ヤフオク画像検索URL':''
            }
            
        print(info)
        data_yahoo.append(info)
    
    #print(data_yahoo)
    return data_yahoo

# ヤフオク画像URL (再検索)
def research_image_yahoo(dict_combine):
    for item in dict_combine:
        #print(item)
        if str(item.get('ヤフオク画像検索URL')) == '':
            search_word = str(item.get('ヤフオク商品名'))
            url = 'https://auctions.yahoo.co.jp/search/search?auccat=&tab_ex=commerce&ei=utf-8&aq=-1&oq=&sc_i=&fr=auc_top&p={}&x=51&y=31'.format(search_word)
            driver.get(url)
            driver.implicitly_wait(3)
            
            print(url)
            try:
                element = driver.find_element_by_class_name('Product__imageData')
                image_url = element.get_attribute('src')
                print(image_url)
                item['ヤフオク画像検索URL'] = 'https://www.google.co.jp/searchbyimage?image_url=' + image_url + '&encoded_image=&image_content=&as_sitesearch=Aliexpress.com&filename=&hl=ja'
            except:
                item['ヤフオク画像検索URL'] = ''
    
    return dict_combine

def main():
  year, month, data_bid = get_info_bid()
  data_yahoo = get_info_yahoo(data_bid)
  # DataFrame を結合
  df_bid = pd.DataFrame(data_bid)
  df_yahoo = pd.DataFrame(data_yahoo)
  #print(df_yahoo)
  #df = pd.merge(df_bid, df_yahoo)
  df_combine = df_bid.join(df_yahoo)
  #print(df_combine)
  # 空白行を削除
  df_combine = df_combine[df_combine['ヤフオクURL'] != '']
  dict_combine = df_combine.to_dict('records')
  # 画像を再検索
  df = pd.DataFrame(research_image_yahoo(dict_combine))
  df = df[df['ヤフオク落札価格'] != '']
  df = df[df['ヤフオク本品'] > border_yahoo]
  df['アリエクURL'] = ''
  df['オプション-1'] = ''
  df['オプション-2'] = ''
  df['オプション-3'] = ''
  df['オプション-4'] = ''
  df = df.reindex(columns=['ベンチマークアカ', 'ヤフオクURL', 'ヤフオク商品名', 'オチサツURL', 'ヤフオク落札価格', 'ヤフオク本品', 'ヤフオク送料','オチサツ画像URL', 'オチサツ画像検索URL', 'ヤフオク画像検索URL', 'アリエクURL', 'オプション-1', 'オプション-2', 'オプション-3', 'オプション-4']) 
  df = df.drop_duplicates(subset='ヤフオク商品名')
  # NG ワード判定
  for NG_word in NG_word_list:
    df = df[~df['ヤフオク商品名'].str.contains(NG_word, na = False)]
  # NG タイトル判定
  for NG_title in NG_title_list:
    df = df[df['ヤフオク商品名'] != NG_title]
  # add work sheet
  workbook_ochisatsu.add_worksheet(title = str(year) + '_' + str(month), rows=500, cols=20)
  worksheet_ochisatsu = workbook_ochisatsu.worksheet(str(year) + '_' + str(month))
  # to spread sheet
  gs_df.set_with_dataframe(worksheet_ochisatsu, df)


if __name__ == '__main__':
  main()


['oceanblue0706', 'hqtbg22504', 'lvyj818aaum', 'muranori416', 'lkanadays', 'dinner_2303yoshii', 'ooqanta5578', 'mailtoyoshihisa0306', 'hy6entertainment', 'ercoa00316', 'ninety_646382023', 'nice_852030843', 'zicyz8', 'rhrjaaww', 'pmatz85385', 'rywmg847', 'srnb358', 'txnnl27680', 'supertenniffer', 'uddfa63102']
検索年を入力
2020
検索月を入力
6
{'ベンチマークアカ': 'oceanblue0706', 'ヤフオクURL': 'https://page.auctions.yahoo.co.jp/jp/auction/e412957187', 'ヤフオク商品名': 'BMW E82 E87 E90E91 LCI E92E93 E60LCI E61LCI E63LCI F01 E84 X1X5X6 E70E71 Z4 E89 LED イカリング', 'オチサツURL': 'http://ochisatsu.com/search/?code=%B4%C1%BB%FA%C8%BD%CA%CC%CD%D1&tl=&id=&u0=oceanblue0706&s=-pr&u1=&d1=&d2=&u2=&p1=&p2=&c=&img=1&20206=8%B7%EE', 'オチサツ画像URL': 'http://ochisatsu.com:8080/e412957187.jpg', 'オチサツ画像検索URL': 'https://www.google.co.jp/searchbyimage?image_url=http://ochisatsu.com:8080/e412957187.jpg&encoded_image=&image_content=&as_sitesearch=Aliexpress.com&filename=&hl=ja'}
{'ベンチマークアカ': 'oceanblue0706', 'ヤフオクURL': 'https://page.auctions.yah

In [1]:
import csv
from time import sleep
import datetime
import pandas as pd
import requests
import pathlib
import random
import re
import glob
import openpyxl
import itertools
from time import sleep
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from itertools import zip_longest
from google.colab import auth
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import gspread
from oauth2client.client import GoogleCredentials
import gspread_dataframe as gs_df

# google settings
auth.authenticate_user()
gc = gspread.authorize(GoogleCredentials.get_application_default())
workbook_research = gc.open_by_url('https://docs.google.com/spreadsheets/d/1BjGJsAQaOTgKwMtD_VVte_3dLpFinQ5wWx8Z_L-HUvY/edit#gid=0')
workbook_result = gc.open_by_url('https://docs.google.com/spreadsheets/d/1q3KAXxPrkbX2AQZ9cgsTT1G3iraeQSRl4eSt4xqwVr4/edit#gid=0')
# selenium settings
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',options=options)

def get_info_ali(dict_df):
  data = []
  for item in dict_df:
    url = str(item['アリエクURL'])
    if url != '':
      driver.get(url)
      driver.implicitly_wait(3)
      # option
      option_list = []
      for key, value in item.items():
        if ('オプション' in key) and (value != ''):
          num = int(value) - 1 
          option_list.append(num)
    
      # Ali Express
      # 広告を閉じる
      try:
          driver.find_element_by_class_name('next-dialog-close').click()
      except:
          pass
      driver.find_element_by_class_name('ng-item-wrap.ng-item.ng-switcher').click()
      # 国選択
      sleep(2)
      driver.find_element_by_class_name('country-selector.switcher-shipto-c.address-selector').click()
      element = driver.find_element_by_class_name('address-select-content')
      country_list = element.find_elements_by_class_name('shipping-text')
      # 発送先を選択(Japan)
      [country.click() for country in country_list if country.text == 'Japan']
      # 選択肢に日本がない場合 
      sleep(2)   
      # 言語を選択(English)
      element = driver.find_element_by_class_name('switcher-currency-c.language-selector')
      element.find_element_by_class_name('select-item').click()
      element_language = element.find_element_by_class_name('notranslate')
      language_list = element_language.find_elements_by_tag_name('li')
      [language.click() for language in language_list if language.text == 'English']
      sleep(2)
      # 通貨を選択(JPY)
      element = driver.find_element_by_class_name('switcher-currency.item.util-clearfix')
      element.find_element_by_class_name('select-item').click()
      element_currency = element.find_element_by_class_name('notranslate')
      currency_list = element_currency.find_elements_by_tag_name('li')
      [print(currency.text) for currency in currency_list]
      # 日本円がある場合
      [currency.click() for currency in currency_list if currency.text == 'JPY ( Japanese Yen )']  
      sleep(2)
      # 決定ボタン
      driver.find_element_by_class_name('ui-button.ui-button-primary.go-contiune-btn').click()
      # 広告を閉じる
      try:
          driver.find_element_by_class_name('next-dialog-close').click()
      except:
          pass
      
      # オプション選択
      optional_elements = driver.find_elements_by_class_name('sku-property')
      for number, option in zip(option_list, optional_elements):
          option_title = option.find_element_by_class_name('sku-title')
          option.find_elements_by_class_name('sku-property-item')[number].click()
          sleep(2)
    # 商品名
      item_title = driver.find_element_by_class_name('product-title-text')
      item_title = item_title.text
      # 辞書に登録
      item['アリエクURL'] = url
      item['アリエク商品名'] = item_title
      # 商品価格
      try:
        item_price = driver.find_element_by_class_name('product-price-value')
        item_price = item_price.text
        print(item_price)
        price = int(re.sub('[^0-9]', '', item_price))    
      except:
        item_price = driver.find_element_by_class_name('current-price')
        item_price = item_price.text
        price = int(re.sub('[^0-9]', '', item_price))
      # 辞書に登録
      item['アリエク本品'] = price
      # 送料
      try:
        item_postage = driver.find_element_by_class_name('product-shipping-price')
        item_postage = item_postage.text
        print(item_postage)
        if ('無料' in item_postage) or ('Free' in item_postage):
            postage = '0'
        else:
            postage = int(re.sub('[^0-9]', '', item_postage))
      except:
          postage = 'Exception'
      # 辞書に登録
      item['アリエク送料'] = postage
      # 総計
      if(postage == '0'):
          amount = price
      elif(type(postage) is int):
          amount = price + postage
      else:
          amount = 'Exception'
      # 辞書に登録
      item['アリエク価格'] = amount
      # 粗利を計算
      item['粗利'] = int(int(item['ヤフオク落札価格']) - int(item['アリエク価格']) - (int(item['ヤフオク落札価格']) * 0.1) + (int(item['アリエク価格']) * 0.03))
      item['粗利率'] = str(round((int(item['粗利']) / int(item['ヤフオク落札価格']) * 100), 2)) + '%'
    # URL がない場合
    else:
      item['アリエクURL'] = ''
      item['アリエク商品名'] = ''
      item['アリエク本品'] = ''
      item['アリエク送料'] = ''
      item['アリエク価格'] = ''
      item['粗利'] = ''
      item['粗利率'] = ''
    print(item)
    data.append(item)
  
  return data

def check_image(data):
  #print(data)
  for item in data: 
    url = item['オチサツ画像検索URL']
    driver.get(url)
    driver.implicitly_wait(3)

    element = driver.find_element_by_class_name('card-section')
    element = element.text

    if '9x9 ピクセル以上の画像を指定してください。' in element:
      item['Google上のアリエクURL'] = item['ヤフオク画像検索URL']
    else:
      item['Google上のアリエクURL'] = item['オチサツ画像検索URL']
  
  return data

         
def main():
  print("年を入力")
  year = int(input())
  print("月を入力")
  month = int(input())

  worksheet_research = workbook_research.worksheet(str(year) + '_' + str(month))
  df = pd.DataFrame(worksheet_research.get_all_values())
  df.columns = list(df.loc[0, :])
  df.drop(0, inplace=True)
  df.reset_index(inplace=True)
  df.drop('index', axis=1, inplace=True)
  dict_df = df.to_dict('records')
  
  data = get_info_ali(dict_df)
  data = check_image(data)
  df = pd.DataFrame(data)
  df = df.reindex(columns=['ベンチマークアカ', 'ヤフオクURL', 'ヤフオク商品名', 'ヤフオク落札価格', 'ヤフオク本品', 'ヤフオク送料', 'アリエクURL', 'アリエク価格', 'アリエク本品', 'アリエク送料', '粗利', '粗利率', 'オチサツ画像URL', 'Google上のアリエクURL', 'アリエクURL', 'オプション-1', 'オプション-2', 'オプション-3', 'オプション-4']) 

  # add work sheet
  workbook_result.add_worksheet(title = str(year) + '_' + str(month), rows=1000, cols=1000)
  worksheet_result = workbook_result.worksheet(str(year) + '_' + str(month))
  # to spread sheet
  gs_df.set_with_dataframe(worksheet_result, df)

if __name__ == '__main__':
  main()



ModuleNotFoundError: ignored